In [3]:
from google.colab import files
uploaded= files.upload();

Saving fused_ship_classifier.pth to fused_ship_classifier.pth


In [4]:
from google.colab import files
uploaded= files.upload();

Saving test.jpg to test.jpg


In [5]:
MODEL_PATH = "/mnt/data/fused_ship_classifier.pth"
IMAGE_PATH = "/mnt/data/test.jpg"


In [6]:
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt


In [7]:
class_names=['Ships']

In [9]:
import os
os.listdir()


['.config', 'test.jpg', 'fused_ship_classifier.pth', 'sample_data']

In [10]:
MODEL_PATH = "/content/fused_ship_classifier.pth"


In [13]:
import torch

MODEL_PATH = "/content/fused_ship_classifier.pth"

model = torch.jit.load(MODEL_PATH, map_location="cpu")
model.eval()


RecursiveScriptModule(
  original_name=FusedEfficientNet
  (model_rgb): RecursiveScriptModule(
    original_name=EfficientNet
    (conv_stem): RecursiveScriptModule(original_name=Conv2d)
    (bn1): RecursiveScriptModule(
      original_name=BatchNormAct2d
      (drop): RecursiveScriptModule(original_name=Identity)
      (act): RecursiveScriptModule(original_name=SiLU)
    )
    (blocks): RecursiveScriptModule(
      original_name=Sequential
      (0): RecursiveScriptModule(
        original_name=Sequential
        (0): RecursiveScriptModule(
          original_name=DepthwiseSeparableConv
          (conv_dw): RecursiveScriptModule(original_name=Conv2d)
          (bn1): RecursiveScriptModule(
            original_name=BatchNormAct2d
            (drop): RecursiveScriptModule(original_name=Identity)
            (act): RecursiveScriptModule(original_name=SiLU)
          )
          (aa): RecursiveScriptModule(original_name=Identity)
          (se): RecursiveScriptModule(
            origina

In [17]:
import torch
from PIL import Image, ImageEnhance
from torchvision import transforms

model = torch.jit.load("/content/fused_ship_classifier.pth", map_location="cpu")
model.eval()


RecursiveScriptModule(
  original_name=FusedEfficientNet
  (model_rgb): RecursiveScriptModule(
    original_name=EfficientNet
    (conv_stem): RecursiveScriptModule(original_name=Conv2d)
    (bn1): RecursiveScriptModule(
      original_name=BatchNormAct2d
      (drop): RecursiveScriptModule(original_name=Identity)
      (act): RecursiveScriptModule(original_name=SiLU)
    )
    (blocks): RecursiveScriptModule(
      original_name=Sequential
      (0): RecursiveScriptModule(
        original_name=Sequential
        (0): RecursiveScriptModule(
          original_name=DepthwiseSeparableConv
          (conv_dw): RecursiveScriptModule(original_name=Conv2d)
          (bn1): RecursiveScriptModule(
            original_name=BatchNormAct2d
            (drop): RecursiveScriptModule(original_name=Identity)
            (act): RecursiveScriptModule(original_name=SiLU)
          )
          (aa): RecursiveScriptModule(original_name=Identity)
          (se): RecursiveScriptModule(
            origina

In [18]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [19]:
def predict_single_image(image_path, threshold=0.6):
    # Load RGB image
    img = Image.open(image_path).convert("RGB")

    # Generate modalities internally
    gray_img = img.convert("L").convert("RGB")
    contrast_img = ImageEnhance.Contrast(img).enhance(2.0)

    # Apply transforms
    rgb = transform(img).unsqueeze(0)
    gray = transform(gray_img).unsqueeze(0)
    contrast = transform(contrast_img).unsqueeze(0)

    # Inference
    with torch.no_grad():
        outputs = model(rgb, gray, contrast)
        probs = torch.softmax(outputs, dim=1)
        confidence, pred_idx = torch.max(probs, dim=1)

    # Class labels (must match training)
    class_names = [
        "Cargo Ship",
        "Cruise Ship",
        "Military Ship",
        "Tanker"
    ]

    # Open-set / ship detection logic
    if confidence.item() < threshold:
        return {
            "label": "Unknown / Not a ship",
            "confidence": confidence.item()
        }

    return {
        "label": class_names[pred_idx.item()],
        "confidence": confidence.item()
    }


In [20]:
result = predict_single_image("/content/test.jpg")

print("Prediction:", result["label"])
print("Confidence:", round(result["confidence"], 3))


Prediction: Cargo Ship
Confidence: 1.0
